In [1]:
import functools
import json
import math
import operator
import re
import sys
from datetime import datetime

import numpy as np
import pandas as pd

from pprint import PrettyPrinter
sys.path.append("../python-src")
from presidential_employment import *

In [2]:
pp = PrettyPrinter(indent=2)

In [3]:
output_dir = "/home/pvh/Documents/code/pvh-forks/presidential-employment-stimulus/data"

### Data structure

Each department has a total budget and total opportunities target. 

The overall programme has outcome targets
1. Jobs created
2. Jobs retained
3. Livelihoods supported

Each department has a "blurb" describing their programme.

Within each department there are multiple programmes that can contribute to each of these targets.

Each programme has a demographic split of outcomes, with gender and youth percentages.

Each programme has a per-province split of outcomes.

Files:

`Consolidated data (Dec) - Presidential Employment Stimulus.xlsx` - December sheet

`Consolidated Presidential Employment Stimulus Reporting Template.xlsx` - January sheet

In [4]:
# dump metric titles (defined in python_src/presidential_employment.py) into metric_title.json
json.dump(metric_titles, open(output_dir + "/metric_titles.json", "w"), indent=2)

In [5]:
# this is where we define the input excel

july_excel = "Dashboard input_PES targets and opportunities per month 130721 Final.xlsx"

august_excel = 'Dashboard input_PES targets and opportunities per month 030821 Final.xlsx'

august_excel2 = 'Dashboard input_PES targets and opportunities per month 090821 Final.xlsx'

august_excel3 = 'Dashboard input_PES targets and opportunities per month 100821 Final.xlsx'

august_excel4 = 'Dashboard input_PES targets and opportunities per month 100821_2 Final.xlsx'

september_excel = 'Dashboard input_PES targets and opportunities September data final 07.10.2021.xlsx'

september_excel2 = 'Dashboard input_PES targets and opportunities September data final 11.10.2021.xlsx'

final_phase_1_excel = 'Copy of Dashboard input_PES targets and opportunities per month 030222 Updated with Implementation.xlsx'

consolidated_dashboard_excel = final_phase_1_excel

phase2_excel = 'Dashboard input_PES targets and opportunities per month Phase 2 Final 4 (1).xlsx'

opportunity_targets_df = [pd.read_excel(
    consolidated_dashboard_excel, sheet_name="Targets", header=None
).fillna(0)]

opportunity_targets_df.append(pd.read_excel(
    phase2_excel, sheet_name="Targets", header=None
).fillna(0))

opportunity_achievements_df = [pd.read_excel(
    consolidated_dashboard_excel, sheet_name="Trends", header=None
).fillna(0)]

opportunity_achievements_df.append(pd.read_excel(
    phase2_excel, sheet_name="Trends", header=None
).fillna(0))

implementation_status_df = [pd.read_excel(
    consolidated_dashboard_excel,
    sheet_name="Implementation status",
    skiprows=2,
    usecols=range(4),
    names=["department", "programme", "status", "detail"],
)]

implementation_status_df.append(pd.read_excel(
    phase2_excel,
    sheet_name="Implementation status",
    skiprows=2,
    usecols=range(4),
    names=["department", "programme", "status", "detail"],
))

for i in range(len(implementation_status_df)):
    implementation_status_df[i].department = implementation_status_df[i].department.fillna(method='pad')
    implementation_status_df[i].detail = implementation_status_df[i].detail.fillna('')

# opportunity_type_df = pd.concat(
#     [opportunity_targets_df.iloc[2:56, 1], opportunity_targets_df.iloc[2:56, 4]], axis=1
# ).set_index(1)


In [6]:
description_df = pd.read_excel(
    phase2_excel,
    sheet_name="Department Descriptions",
    names=['key', 'lead', 'paragraph', 'Data captured until'],
    usecols=range(4),
    index_col=0
).dropna()
description_df

,lead,paragraph,Data captured until
key,,,
Overview,Building a society that works: Continuty and c...,The Presidential Employment Stimulus was estab...,31-01-2022
DALRRD,Expanding support to farmers and protecting fo...,The pandemic illustrated the vulnerability of ...,31-01-2022
DFFE,,"The work undertaken in environmental, forestry...",31-01-2022
DSAC,Preserving our heritage-support to the Distric...,"In Phase One, the focus of the programme was o...",31-01-2022
COGTA,New approaches to solid waste management,Through the Municipal Infrastructure Support A...,31-01-2022
DTIC,Social Employment Fund - work for the common good,Phase Two is supporting the development of a ‘...,31-01-2022
DoH,Primary Health Care is at the frontline of the...,As the world responded to the COVID-19 pandemi...,31-01-2022
DSI,Science skills support communities,The DSI will continue to support a range of ci...,31-01-2022
NT,Innovation in Public Employment Programmes in ...,National Treasury’s Neighbourhood Development ...,31-01-2022


In [7]:
phase1_departments = set(
    pd.read_excel(consolidated_dashboard_excel, sheet_name="Targets", skiprows=1)
    .loc[:, "Department"]
    .dropna()
    .iloc[:-1]
)

phase2_departments = set(
    pd.read_excel(phase2_excel, sheet_name="Targets", skiprows=1)
    .loc[:, "Department"]
    .dropna()
    .iloc[:-1]
)
    
department_names = list(set(phase1_departments).union(phase2_departments))

# programmes_names = (
#     pd.read_excel(consolidated_dashboard_excel, sheet_name="Targets", skiprows=1)
#     .loc[:, "Programme"]
#     .dropna()
#     .iloc[:-1]
# )

# programmes_by_department = {}
# for row in (
#     pd.read_excel(consolidated_dashboard_excel, sheet_name="Targets", skiprows=1)
#     .loc[:, "Department":"Programme"]
#     .iloc[
#         :-1,
#     ]
#     .iterrows()
# ):
#     if not pd.isna(row[1][0]):
#         department_name = row[1][0]
#     programme_name = row[1][1]
#     programme_names = programmes_by_department.get(department_name, [])
#     programme_names.append(programme_name)
#     programmes_by_department[department_name] = programme_names
# list(department_names)

In [8]:
leads = description_df.lead.to_dict()
paragraphs = description_df.paragraph.to_dict()

In [9]:
targets_df = [pd.read_excel(
    consolidated_dashboard_excel,
    sheet_name="Targets",
    skiprows=1,
    usecols=list(range(6)),
    names=["department", "programme", "target", "unk", "section", "display_name"],
).drop("unk", axis=1)]
targets_df.append(pd.read_excel(
    phase2_excel,
    sheet_name="Targets",
    skiprows=1,
    usecols=list(range(6)),
    names=["department", "programme", "target", "unk", "section", "display_name"],
))

for i in range(len(targets_df)):
    targets_df[i].department = targets_df[i].department.fillna(method="pad")
    targets_df[i].section = targets_df[i].section.fillna(method="pad")

In [10]:
trends_df = [pd.read_excel(
    consolidated_dashboard_excel,
    sheet_name="Trends",
    skiprows=5,
    usecols=list(range(total_achievement_column[0]+1)),
)]
trends_df.append(pd.read_excel(
    phase2_excel,
    sheet_name="Trends",
    skiprows=4,
    usecols=list(range(total_achievement_column[1]+1)),
))
for i in range(len(trends_df)):
    trends_df[i].columns = [c.lower() for c in trends_df[i].columns]
    trends_df[i].department = trends_df[i].department.fillna(method="pad")
    trends_df[i] = trends_df[i].fillna(0)
    if i == 1:
        trends_df[i] = trends_df[i].drop('oct', axis=1)

In [11]:
provincial_df = [pd.read_excel(
    consolidated_dashboard_excel,
    sheet_name="Provincial (beneficiaries)",
    skiprows=4,
    usecols=list(range(12)),
)]
provincial_df.append(pd.read_excel(
    phase2_excel,
    sheet_name="Provincial (beneficiaries)",
    skiprows=4,
    usecols=list(range(12)),
))

for i in range(len(provincial_df)):
    provincial_df[i].columns = [
        c.lower().replace(" ", "_").replace("-", "_") for c in provincial_df[i].columns
    ]
    provincial_df[i].department = provincial_df[i].department.fillna(method="pad")
    provincial_df[i] = provincial_df[i].fillna(0)

In [12]:
demographic_df = [pd.read_excel(
    consolidated_dashboard_excel,
    sheet_name="Demographic data",
    skiprows=8,
    usecols=list(range(9)),
)]

demographic_df.append(pd.read_excel(
    phase2_excel,
    sheet_name="Demographic data",
    skiprows=9,
    usecols=list(range(9)),
))

for i in range(len(demographic_df)):
    demographic_df[i].columns = [
        c.lower().replace(" ", "_").replace("%", "perc").replace('no.', 'no') for c in demographic_df[i].columns
    ]
    demographic_df[i].department = demographic_df[i].department.fillna(method="pad")
# demographic_df = demographic_df.fillna(0)

In [13]:
# the line below was removed because it is better to special-case those programmes
# targets_df = targets_df.fillna(-1)  # this is to ensure that targets are -1 when no value is available


## 

# Top level structure

In [14]:
def add_or_replace(departments, department):
    # if a department with sheet_name exists in the list, replace it with the new department, else append to list
    for i, el in enumerate(departments):
        if el.sheet_name == department.sheet_name:
            departments[i] = department
            break
    else:
        departments.append(department)
    return departments

In [15]:
def make_dim(dim_type, lookup_type, df, col_start, col_end, key_lookup):
    row = df.loc[(df.department == department_name) & (df.programme == programme_name)]
    values = []
    if len(df.loc[(df.department == department_name) & (df.programme == programme_name)]) == 0:
        data_missing = True
    else:
        nonzero = False
        for key in list(row)[col_start:col_end]:
            value = int(row.loc[:, key])
            if value > 0:
                nonzero = True
            values.append(MetricValue(key=key_lookup(key), value=value))
        if not nonzero:
            data_missing = True
            values = []
        else:
            data_missing = False

    dim = Dimension(
        name=metric_titles[section_abbrev_to_name[section]][
            MetricTypeEnum.count.name + "_" + dim_type
        ],
        lookup=dim_type,
        viz=lookup_type,
        values=values,
        data_missing=data_missing,
    )
    return dim

In [37]:
all_data_departments=[]

desc_abbrevs = {"DoH": "DoH",
                "Tourism": "Tourism ",
                "DPWI": "DPWI ",
                "DCOGTA": "COGTA" }  # deal with special cases in description lookup
departments = {}
for department_name in department_names:
    phases = []
    for phase_num in range(number_of_phases):
        if phase_num == 0 and (not department_name in phase1_departments):
            continue
        elif phase_num == 1 and (not department_name in phase2_departments):
            continue
        department_implementation_details = []
        target_section = Section(
            name=section_titles[SectionEnum.targets.name],
            section_type=SectionEnum.targets.name,
            metrics=[
                Metric(
                    name=metric_titles[SectionEnum.targets.name][
                        MetricTypeEnum.currency.name
                    ],
                    metric_type=MetricTypeEnum.currency.name,
                    value_target=department_budget_targets[phase_num][department_name],
                    value=-1,
                    dimensions=[],
                ),
                Metric(
                    name=metric_titles[SectionEnum.targets.name][MetricTypeEnum.count.name],
                    metric_type=MetricTypeEnum.count.name,
                    value_target=targets_df[phase_num].loc[
                        targets_df[phase_num].department == department_name
                    ].target.sum(),  # overall target of beneficiaries
                    value=trends_df[phase_num].loc[trends_df[phase_num].department == department_name]
                    .iloc[:, -1]
                    .sum(),  # get the achievement by summing the last column in trends
                    dimensions=[],
                ),
            ],
        )
        
        sections = [target_section]
        for section in ["CRE", "LIV", "RET"]:  # TODO: support CAT - Catalytic Interventions
            programme_names = list(
                targets_df[phase_num].loc[
                    (targets_df[phase_num].section == section)
                    & (targets_df[phase_num].department == department_name)
                ].programme
            )
            if phase_num == 0 and section == 'CRE' and department_name == 'Agriculture, Land Reform and Rural Development':
                # this does not have a target so needs to be added manually
                programme_names += ['Graduate verifiers']
            metrics = []

            for programme_name in programme_names:
                if department_name == 'Public Works and Infrastructure' and programme_name == 'Project Administrators':
                    # this programme is mentioned in Targets and has a line in Implementation Status but has no other data
                    continue
                imp_status_row = implementation_status_df[phase_num].loc[
                    (implementation_status_df[phase_num].department == department_name)
                    & (implementation_status_df[phase_num].programme == programme_name)
                ]
                if len(imp_status_row) == 0 or pd.isna(imp_status_row.status.iloc[0]):
                    imp_detail = None
                else:
                    imp_detail = ImplementationDetail(
                        programme_name=programme_name,
                        status=implementation_status_to_enum[imp_status_row.status.iloc[0].strip()],
                        detail=imp_status_row.detail.iloc[0].strip(),
                    )
                if (
                    department_name == "Public Works and Infrastructure"
                    and programme_name
                    == "Graduate programmes (Property Management Trading Entity)"
                ) or (
                    department_name == "Agriculture, Land Reform and Rural Development"
                    and programme_name == "Subsistence producer relief fund"
                ):
                    department_implementation_details.append(imp_detail)
                    continue  # these programmes have no detailed metrics
                else:
                    try:
                        # collect detailed metrics for programme
                        dimensions = []
                        time_dimension_row = trends_df[phase_num].loc[
                            (trends_df[phase_num].department == department_name)
                            & (trends_df[phase_num].programme == programme_name)
                        ]
                        if department_name == 'Cooperative Governance' and programme_name == 'Mainstream labour intensive construction methods':
                            print("got here", time_dimension_row)

                        dimensions.append(make_dim(LookupTypeEnum.province.name, VizTypeEnum.bar.name, provincial_df[phase_num], 2, -1, lambda key: province_header_to_abbrev[key]))
                        dimensions.append(make_dim(LookupTypeEnum.time.name, VizTypeEnum.line.name, trends_df[phase_num], 2, None, lambda key: month_lookup[phase_num][key]))

                        demographic_row = demographic_df[phase_num].loc[
                            (demographic_df[phase_num].department == department_name)
                            & (demographic_df[phase_num].programme == programme_name)
                        ]

                        values = []
                        if len(demographic_row) == 0:
                            data_missing = True
                        else:
                            male_perc = demographic_row.loc[:, "perc_male"].iloc[0]
                            female_perc = demographic_row.loc[:, "perc_female"].iloc[0]
                            if male_perc + female_perc == 0:
                                data_missing = True
                            else:
                                values=[
                                    MetricValue(
                                        key=GenderEnum.Male.name,
                                        value=male_perc,
                                    ),
                                    MetricValue(
                                        key=GenderEnum.Female.name,
                                        value=female_perc,
                                    ),
                                ]
                                if male_perc + female_perc != 1.0:
                                    print("M/F PERC PROBLEM:", department_name, programme_name, phase_num, male_perc, female_perc, male_perc + female_perc)
                                data_missing=False

                        gender_dim = Dimension(
                            name=metric_titles[section_abbrev_to_name[section]][
                                MetricTypeEnum.count.name + "_gender"
                            ],
                            lookup=LookupTypeEnum.gender.name,
                            viz=VizTypeEnum.two_value.name,
                            values=values,
                            data_missing=data_missing
                        )
                        dimensions.append(gender_dim)

                        values = []
                        if len(demographic_row) == 0:
                            data_missing = True
                        else:
                            age_perc = demographic_row.loc[:, "perc_youth"].iloc[0]
                            if age_perc == 0:
                                data_missing = True
                                values = []
                            else:
                                values=[
                                    MetricValue(
                                        key="18-35",
                                        value=age_perc,
                                    )
                                ]
                                data_missing = False
                        youth_dim = Dimension(
                            name=metric_titles[section_abbrev_to_name[section]][
                                MetricTypeEnum.count.name + "_age"
                            ],
                            lookup=LookupTypeEnum.age.name,
                            viz=VizTypeEnum.percentile.name,
                            values=values,
                            data_missing=data_missing
                        )
                        dimensions.append(youth_dim)

#                         # TODO: Rationalise this - disabled and military vets share a lot of code
#                         if phase_num == 0:
#                             disabled = demographic_row.no_disability.iloc[0]
#                             if disabled > 0:
#                                 disabled_dim = Dimension(
#                                     name=metric_titles[section_abbrev_to_name[section]][MetricTypeEnum.count.name + '_disabled'],
#                                     lookup=LookupTypeEnum.disabled.name,
#                                     viz=VizTypeEnum.count.name,
#                                     values=[MetricValue(key='disabled', value=disabled)]
#                                 )
#                                 dimensions.append(disabled_dim)

#                             # military_vets = demographic_row.no_military_veterans.iloc[0]
#                             # if military_vets > 0:
#                             #     mv_dim = Dimension(
#                             #         name=metric_titles[section_abbrev_to_name[section]][MetricTypeEnum.count.name + '_vets'],
#                             #         lookup=LookupTypeEnum.vets.name,
#                             #         viz=VizTypeEnum.count.name,
#                             #         values=[MetricValue(key='vets', value=military_vets)]
#                             #     )
#                             #     dimensions.append(mv_dim)
#                         elif phase_num == 1:                            
#                             perc_disabled = demographic_row.perc_disability.iloc[0]
#                             perc_not_disabled = 1 - perc_disabled
#                             if perc_disabled > 0:
#                                 disabled_dim = Dimension(
#                                     name=metric_titles[section_abbrev_to_name[section]][MetricTypeEnum.count.name + '_disabled'],
#                                     lookup=LookupTypeEnum.disabled.name,
#                                     viz=VizTypeEnum.two_value.name,
#                                     values=[MetricValue(key='disabled', value=perc_disabled), MetricValue(key='not disabled', value=perc_not_disabled)]
#                                 )
#                                 dimensions.append(disabled_dim)

                        total_value = int(time_dimension_row.iloc[:,-1].iloc[0])

                        target_row = targets_df[phase_num].fillna(0).loc[
                                (targets_df[phase_num].department == department_name)
                                & (targets_df[phase_num].programme == programme_name)
                            ].target
                        if len(target_row) == 0:
                            # e.g. Graduate verifiers programme doesn't have a target
                            target = -1
                        else:
                            target = target_row.iloc[0]
                        programme_metric = Metric(
                            name=programme_name,
                            metric_type=MetricTypeEnum.count.name,
                            value=total_value,
                            value_target=target,
                            dimensions=dimensions,
                            implementation_detail=imp_detail,
                        )
                        metrics.append(programme_metric)
                    except IndexError as e:
                        print("IndexError on", section, department_name, programme_name, str(e))

            sections.append(
                Section(
                    name=section_titles[section_abbrev_to_name[section]],
                    section_type=section_abbrev_to_name[section],
                    metrics=metrics,
                )
            )
        abbrev = department_name_to_abbreviation[department_name]
        
        month_info = description_df.loc[
            desc_abbrevs.get(abbrev, abbrev), "Data captured until"
        ]
        try:
            month = month_info.strftime('%Y%m')
        except AttributeError as e:
            month_parts = month_info.split('-')
            month = month_parts[2] + month_parts[1]
        phase = Phase(
                phase_num=phase_num,
                month=month,
                sections=sections,
                target_lines=[],
                achievement_lines=[],
                implementation_details=department_implementation_details,
                beneficiaries = []
            )
        phases.append(phase)
    departments[department_name] = Department(
        name=department_name,
        sheet_name=abbrev,
        lead=leads[desc_abbrevs.get(abbrev, abbrev)],
        paragraph=paragraphs[desc_abbrevs.get(abbrev, abbrev)],
        phases=phases
    )
    
for name in sorted(departments.keys()):
    all_data_departments.append(departments[name])    

M/F PERC PROBLEM: Department of Women, Youth and Persons with Disabilities PYEI/ Youth Enterprise Support Fund  1 0.52 0.46 0.98
M/F PERC PROBLEM: Transport Rural roads maintenance 0 0.49 0.34 0.8300000000000001
M/F PERC PROBLEM: Agriculture, Land Reform and Rural Development Vegetables and Fruits 0 0.1 0.31 0.41000000000000003
M/F PERC PROBLEM: Agriculture, Land Reform and Rural Development Maize/soya/sugar/other production 0 0.033 0.09 0.123
M/F PERC PROBLEM: Agriculture, Land Reform and Rural Development Poultry: Layers and Boilers 0 0.08 0.19 0.27
M/F PERC PROBLEM: Agriculture, Land Reform and Rural Development Small livestock 0 0.06 0.08 0.14
M/F PERC PROBLEM: Agriculture, Land Reform and Rural Development Large livestock 0 0.03 0.04 0.07
M/F PERC PROBLEM: Science and Innovation Water Graduate Programme 1 0.21 0.46 0.67
M/F PERC PROBLEM: Tourism Support to 40 provincial state-owned tourism attractions 1 0.48 0.42 0.8999999999999999
M/F PERC PROBLEM: Basic Education Education Assis

In [17]:
# number_of_phases = 2

# all_data.departments=[]

# desc_abbrevs = {"DoH": "DOH"}  # deal with special cases in description lookup
# departments = {}
# for department_name in department_names:
#     if department_name in phase1_departments:
#         phase1 = True
#     if department_name in phase2_departments:
#         phase2 = True
#     for phase_num in range(number_of_phases):
#         department_implementation_details = []
#         target_section = Section(
#             name=section_titles[SectionEnum.targets.name],
#             section_type=SectionEnum.targets.name,
#             metrics=[
#                 Metric(
#                     name=metric_titles[SectionEnum.targets.name][
#                         MetricTypeEnum.currency.name
#                     ],
#                     metric_type=MetricTypeEnum.currency.name,
#                     value_target=department_budget_targets[department_name], ## TODO: update this for multiple phases
#                     value=-1,
#                     dimensions=[],
#                 ),
#                 Metric(
#                     name=metric_titles[SectionEnum.targets.name][MetricTypeEnum.count.name],
#                     metric_type=MetricTypeEnum.count.name,
#                     value_target=targets_df[phase_num].loc[
#                         targets_df[phase_num].department == department_name
#                     ].target.sum(),  # overall target of beneficiaries
#                     value=trends_df.loc[trends_df.department == department_name]
#                     .iloc[:, -1]
#                     .sum(),  # get the achievement by summing the last column in trends
#                     dimensions=[],
#                 ),
#             ],
#         )
#         sections = [target_section]
#         for section in ["CRE", "LIV", "RET"]:
#             programme_names = list(
#                 targets_df.loc[
#                     (targets_df.section == section)
#                     & (targets_df.department == department_name)
#                 ].programme
#             )
#             if section == 'CRE' and department_name == 'Agriculture, Land Reform and Rural Development':
#                 # this does not have a target so needs to be added manually
#                 programme_names += ['Graduate verifiers']
#             metrics = []

#             for programme_name in programme_names:
#                 if department_name == 'Public Works and Infrastructure' and programme_name == 'Project Administrators':
#                     # this programme is mentioned in Targets and has a line in Implementation Status but has no other data
#                     continue
#                 imp_status_row = implementation_status_df.loc[
#                     (implementation_status_df.department == department_name)
#                     & (implementation_status_df.programme == programme_name)
#                 ]
#                 if len(imp_status_row) == 0 or pd.isna(imp_status_row.status.iloc[0]):
#                     imp_detail = None
#                 else:
#                     imp_detail = ImplementationDetail(
#                         programme_name=programme_name,
#                         status=implementation_status_to_enum[imp_status_row.status.iloc[0].strip()],
#                         detail=imp_status_row.detail.iloc[0].strip(),
#                     )

#                 if (
#                     department_name == "Public Works and Infrastructure"
#                     and programme_name
#                     == "Graduate programmes (Property Management Trading Entity)"
#                 ) or (
#                     department_name == "Agriculture, Land Reform and Rural Development"
#                     and programme_name == "Subsistence producer relief fund"
#                 ):
#                     department_implementation_details.append(imp_detail)
#                     continue  # these programmes have no detailed metrics
#                 else:
#                     try:
#                         # collect detailed metrics for programme
#                         dimensions = []
#                         time_dimension_row = trends_df.loc[
#                             (trends_df.department == department_name)
#                             & (trends_df.programme == programme_name)
#                         ]

#                         dimensions.append(make_dim(LookupTypeEnum.province.name, VizTypeEnum.bar.name, provincial_df[phase_num], 2, -1, lambda key: province_header_to_abbrev[key]))
#                         dimensions.append(make_dim(LookupTypeEnum.time.name, VizTypeEnum.line.name, trends_df[phase_num], 2, None, lambda key: month_lookup[phase_num][key]))

#                         demographic_row = demographic_df.loc[
#                             (demographic_df.department == department_name)
#                             & (demographic_df.programme == programme_name)
#                         ]

#                         values = []
#                         if len(demographic_row) == 0:
#                             data_missing = True
#                         else:
#                             male_perc = demographic_row.loc[:, "perc_male"].iloc[0]
#                             female_perc = demographic_row.loc[:, "perc_female"].iloc[0]
#                             if male_perc + female_perc == 0:
#                                 data_missing = True
#                             else:
#                                 values=[
#                                     MetricValue(
#                                         key=GenderEnum.Male.name,
#                                         value=male_perc,
#                                     ),
#                                     MetricValue(
#                                         key=GenderEnum.Female.name,
#                                         value=female_perc,
#                                     ),
#                                 ]
#                                 if male_perc + female_perc != 1.0:
#                                     print(programme_name, male_perc, female_perc, male_perc + female_perc)
#                                 data_missing=False

#                         gender_dim = Dimension(
#                             name=metric_titles[section_abbrev_to_name[section]][
#                                 MetricTypeEnum.count.name + "_gender"
#                             ],
#                             lookup=LookupTypeEnum.gender.name,
#                             viz=VizTypeEnum.two_value.name,
#                             values=values,
#                             data_missing=data_missing
#                         )
#                         dimensions.append(gender_dim)

#                         values = []
#                         if len(demographic_row) == 0:
#                             data_missing = True
#                         else:
#                             age_perc = demographic_row.loc[:, "perc_youth"].iloc[0]
#                             if age_perc == 0:
#                                 data_missing = True
#                                 values = []
#                             else:
#                                 values=[
#                                     MetricValue(
#                                         key="18-35",
#                                         value=age_perc,
#                                     )
#                                 ]
#                                 data_missing = False
#                         youth_dim = Dimension(
#                             name=metric_titles[section_abbrev_to_name[section]][
#                                 MetricTypeEnum.count.name + "_age"
#                             ],
#                             lookup=LookupTypeEnum.age.name,
#                             viz=VizTypeEnum.percentile.name,
#                             values=values,
#                             data_missing=data_missing
#                         )
#                         dimensions.append(youth_dim)

#                         # TODO: Rationalise this - disabled and military vets share a lot of code
#                         disabled = demographic_row.no_disability.iloc[0]
#                         if disabled > 0:
#                             disabled_dim = Dimension(
#                                 name=metric_titles[section_abbrev_to_name[section]][MetricTypeEnum.count.name + '_disabled'],
#                                 lookup=LookupTypeEnum.disabled.name,
#                                 viz=VizTypeEnum.count.name,
#                                 values=[MetricValue(key='disabled', value=disabled)]
#                             )
#                             dimensions.append(disabled_dim)

#                         military_vets = demographic_row.no_military_veterans.iloc[0]
#                         if military_vets > 0:
#                             mv_dim = Dimension(
#                                 name=metric_titles[section_abbrev_to_name[section]][MetricTypeEnum.count.name + '_vets'],
#                                 lookup=LookupTypeEnum.vets.name,
#                                 viz=VizTypeEnum.count.name,
#                                 values=[MetricValue(key='vets', value=military_vets)]
#                             )
#                             dimensions.append(mv_dim)


#                         total_value = int(time_dimension_row.iloc[:,-1].iloc[0])
#                         target_row = targets_df.fillna(0).loc[
#                                 (targets_df.department == department_name)
#                                 & (targets_df.programme == programme_name)
#                             ].target
#                         if len(target_row) == 0:
#                             # e.g. Graduate verifiers programme doesn't have a target
#                             target = -1
#                         else:
#                             target = target_row.iloc[0]
#                         programme_metric = Metric(
#                             name=programme_name,
#                             metric_type=MetricTypeEnum.count.name,
#                             value=total_value,
#                             value_target=target,
#                             dimensions=dimensions,
#                             implementation_detail=imp_detail,
#                         )
#                         metrics.append(programme_metric)
#                     except IndexError as e:
#                         print("IndexError on", section, department_name, programme_name, str(e))

#             sections.append(
#                 Section(
#                     name=section_titles[section_abbrev_to_name[section]],
#                     section_type=section_abbrev_to_name[section],
#                     metrics=metrics,
#                 )
#             )
#         abbrev = department_name_to_abbreviation[department_name]
#         month = description_df.loc[
#             desc_abbrevs.get(abbrev, abbrev), "Data captured until"
#         ].strftime("%Y%m")
#         phase1 = Phase(
#                 month=month,
#                 sections=sections,
#                 target_lines=[],
#                 achievement_lines=[],
#                 implementation_details=department_implementation_details,
#                 beneficiaries = []
#             )
#         departments[department_name] = Department(
#             name=department_name,
#             sheet_name=abbrev,
#             lead=leads[desc_abbrevs.get(abbrev, abbrev)],
#             paragraph=paragraphs[desc_abbrevs.get(abbrev, abbrev)],
#             phases=[phase1]
#         )


In [18]:
# all_data.departments=[]

# desc_abbrevs = {"DoH": "DOH"}  # deal with special cases in description lookup
# departments = {}
# for department_name in department_names:
#     if department_name in phase1_departments:
#         phase1 = True
#     if department_name in phase2_departments:
#         phase2 = True
#     department_implementation_details = []
#     target_section = Section(
#         name=section_titles[SectionEnum.targets.name],
#         section_type=SectionEnum.targets.name,
#         metrics=[
#             Metric(
#                 name=metric_titles[SectionEnum.targets.name][
#                     MetricTypeEnum.currency.name
#                 ],
#                 metric_type=MetricTypeEnum.currency.name,
#                 value_target=department_budget_targets[department_name],
#                 value=-1,
#                 dimensions=[],
#             ),
#             Metric(
#                 name=metric_titles[SectionEnum.targets.name][MetricTypeEnum.count.name],
#                 metric_type=MetricTypeEnum.count.name,
#                 value_target=targets_df.loc[
#                     targets_df.department == department_name
#                 ].target.sum(),  # overall target of beneficiaries
#                 value=trends_df.loc[trends_df.department == department_name]
#                 .iloc[:, -1]
#                 .sum(),  # get the achievement by summing the last column in trends
#                 dimensions=[],
#             ),
#         ],
#     )
#     sections = [target_section]
#     for section in ["CRE", "LIV", "RET"]:
#         programme_names = list(
#             targets_df.loc[
#                 (targets_df.section == section)
#                 & (targets_df.department == department_name)
#             ].programme
#         )
#         if section == 'CRE' and department_name == 'Agriculture, Land Reform and Rural Development':
#             # this does not have a target so needs to be added manually
#             programme_names += ['Graduate verifiers']
#         metrics = []

#         for programme_name in programme_names:
#             if department_name == 'Public Works and Infrastructure' and programme_name == 'Project Administrators':
#                 # this programme is mentioned in Targets and has a line in Implementation Status but has no other data
#                 continue
#             imp_status_row = implementation_status_df.loc[
#                 (implementation_status_df.department == department_name)
#                 & (implementation_status_df.programme == programme_name)
#             ]
#             if len(imp_status_row) == 0 or pd.isna(imp_status_row.status.iloc[0]):
#                 imp_detail = None
#             else:
#                 imp_detail = ImplementationDetail(
#                     programme_name=programme_name,
#                     status=implementation_status_to_enum[imp_status_row.status.iloc[0].strip()],
#                     detail=imp_status_row.detail.iloc[0].strip(),
#                 )

#             if (
#                 department_name == "Public Works and Infrastructure"
#                 and programme_name
#                 == "Graduate programmes (Property Management Trading Entity)"
#             ) or (
#                 department_name == "Agriculture, Land Reform and Rural Development"
#                 and programme_name == "Subsistence producer relief fund"
#             ):
#                 department_implementation_details.append(imp_detail)
#                 continue  # these programmes have no detailed metrics
#             else:
#                 try:
#                     # collect detailed metrics for programme
#                     dimensions = []
#                     time_dimension_row = trends_df.loc[
#                         (trends_df.department == department_name)
#                         & (trends_df.programme == programme_name)
#                     ]

#                     dimensions.append(make_dim(LookupTypeEnum.province.name, VizTypeEnum.bar.name, provincial_df[phase_num], 2, -1, lambda key: province_header_to_abbrev[key]))
#                     dimensions.append(make_dim(LookupTypeEnum.time.name, VizTypeEnum.line.name, trends_df[phase_num], 2, None, lambda key: month_lookup[phase_num][key]))

#                     demographic_row = demographic_df.loc[
#                         (demographic_df.department == department_name)
#                         & (demographic_df.programme == programme_name)
#                     ]

#                     values = []
#                     if len(demographic_row) == 0:
#                         data_missing = True
#                     else:
#                         male_perc = demographic_row.loc[:, "perc_male"].iloc[0]
#                         female_perc = demographic_row.loc[:, "perc_female"].iloc[0]
#                         if male_perc + female_perc == 0:
#                             data_missing = True
#                         else:
#                             values=[
#                                 MetricValue(
#                                     key=GenderEnum.Male.name,
#                                     value=male_perc,
#                                 ),
#                                 MetricValue(
#                                     key=GenderEnum.Female.name,
#                                     value=female_perc,
#                                 ),
#                             ]
#                             if male_perc + female_perc != 1.0:
#                                 print(programme_name, male_perc, female_perc, male_perc + female_perc)
#                             data_missing=False

#                     gender_dim = Dimension(
#                         name=metric_titles[section_abbrev_to_name[section]][
#                             MetricTypeEnum.count.name + "_gender"
#                         ],
#                         lookup=LookupTypeEnum.gender.name,
#                         viz=VizTypeEnum.two_value.name,
#                         values=values,
#                         data_missing=data_missing
#                     )
#                     dimensions.append(gender_dim)

#                     values = []
#                     if len(demographic_row) == 0:
#                         data_missing = True
#                     else:
#                         age_perc = demographic_row.loc[:, "perc_youth"].iloc[0]
#                         if age_perc == 0:
#                             data_missing = True
#                             values = []
#                         else:
#                             values=[
#                                 MetricValue(
#                                     key="18-35",
#                                     value=age_perc,
#                                 )
#                             ]
#                             data_missing = False
#                     youth_dim = Dimension(
#                         name=metric_titles[section_abbrev_to_name[section]][
#                             MetricTypeEnum.count.name + "_age"
#                         ],
#                         lookup=LookupTypeEnum.age.name,
#                         viz=VizTypeEnum.percentile.name,
#                         values=values,
#                         data_missing=data_missing
#                     )
#                     dimensions.append(youth_dim)

#                     # TODO: Rationalise this - disabled and military vets share a lot of code
#                     disabled = demographic_row.no_disability.iloc[0]
#                     if disabled > 0:
#                         disabled_dim = Dimension(
#                             name=metric_titles[section_abbrev_to_name[section]][MetricTypeEnum.count.name + '_disabled'],
#                             lookup=LookupTypeEnum.disabled.name,
#                             viz=VizTypeEnum.count.name,
#                             values=[MetricValue(key='disabled', value=disabled)]
#                         )
#                         dimensions.append(disabled_dim)
                    
#                     military_vets = demographic_row.no_military_veterans.iloc[0]
#                     if military_vets > 0:
#                         mv_dim = Dimension(
#                             name=metric_titles[section_abbrev_to_name[section]][MetricTypeEnum.count.name + '_vets'],
#                             lookup=LookupTypeEnum.vets.name,
#                             viz=VizTypeEnum.count.name,
#                             values=[MetricValue(key='vets', value=military_vets)]
#                         )
#                         dimensions.append(mv_dim)
                                                                                
                        
#                     total_value = int(time_dimension_row.iloc[:,-1].iloc[0])
#                     target_row = targets_df.fillna(0).loc[
#                             (targets_df.department == department_name)
#                             & (targets_df.programme == programme_name)
#                         ].target
#                     if len(target_row) == 0:
#                         # e.g. Graduate verifiers programme doesn't have a target
#                         target = -1
#                     else:
#                         target = target_row.iloc[0]
#                     programme_metric = Metric(
#                         name=programme_name,
#                         metric_type=MetricTypeEnum.count.name,
#                         value=total_value,
#                         value_target=target,
#                         dimensions=dimensions,
#                         implementation_detail=imp_detail,
#                     )
#                     metrics.append(programme_metric)
#                 except IndexError as e:
#                     print("IndexError on", section, department_name, programme_name, str(e))

#         sections.append(
#             Section(
#                 name=section_titles[section_abbrev_to_name[section]],
#                 section_type=section_abbrev_to_name[section],
#                 metrics=metrics,
#             )
#         )
#     abbrev = department_name_to_abbreviation[department_name]
#     month = description_df.loc[
#         desc_abbrevs.get(abbrev, abbrev), "Data captured until"
#     ].strftime("%Y%m")
#     phase1 = Phase(
#             month=month,
#             sections=sections,
#             target_lines=[],
#             achievement_lines=[],
#             implementation_details=department_implementation_details,
#             beneficiaries = []
#         )
#     departments[department_name] = Department(
#         name=department_name,
#         sheet_name=abbrev,
#         lead=leads[desc_abbrevs.get(abbrev, abbrev)],
#         paragraph=paragraphs[desc_abbrevs.get(abbrev, abbrev)],
#         phases=[phase1]
#     )

# for name in sorted(departments.keys()):
#     all_data.departments.append(departments[name])
    
# # print(all_data.to_json(indent=2))

In [19]:
total_male = [0] * number_of_phases
total_female = [0] * number_of_phases
total_unknown_gender = [0] * number_of_phases
total_beneficiaries = [0] * number_of_phases
total_youth = [0] * number_of_phases
total_unknown_youth = [0] * number_of_phases
total_provincial = {}
total_unknown_province = [0] * number_of_phases

for department in all_data_departments:

    for abbreviation in province_abbreviations:
        total_provincial[abbreviation] = [0] * number_of_phases
    for phase in department.phases:
        phase_num = phase.phase_num
        department_male = department_female = department_beneficiaries = 0
        for section in phase.sections:
            for metric in section.metrics:
                if section.section_type == SectionEnum.targets.name and metric.name == "Beneficiaries":
                    total_beneficiaries[phase_num] += metric.value
                    # print("adding", department.sheet_name, phase_num+1, metric.value, total_beneficiaries)
                    department_beneficiaries = metric.value
                    continue
                if metric.value == -1:
                    continue
                total_value = metric.value
                gender_found = False
                age_found = False
                province_found = False
                for dimension in metric.dimensions:
                    if dimension.data_missing:
                        continue
                    if dimension.lookup == LookupTypeEnum.gender.name:
                        gender_found = True
                        for value in dimension.values:
                            if value.key == 'Male':
                                department_male += total_value * value.value
                                total_male[phase_num] += total_value * value.value
                            elif value.key == 'Female':
                                department_female += total_value * value.value
                                total_female[phase_num] += total_value * value.value
                    elif dimension.lookup == LookupTypeEnum.age.name:
                        age_found = True
                        youth_value = dimension.values[0].value
                        total_youth[phase_num] += youth_value * total_value
                    elif dimension.lookup == LookupTypeEnum.province.name:
                        province_found = True
                        for value in dimension.values:
                            total_provincial[value.key][phase_num] += value.value
                if metric.value <= 0:
                    continue
                if not gender_found:
                    total_unknown_gender[phase_num] += metric.value
                if not age_found:
                    total_unknown_youth[phase_num] += metric.value
                if not province_found:
                    total_unknown_province[phase_num] += metric.value
        # print(department.name, phase_num+1, total_beneficiaries, total_unknown_gender, round(total_unknown_gender / total_beneficiaries, 2), 
        #       total_unknown_youth, round(total_unknown_youth / total_beneficiaries, 2), 
        #       total_unknown_province, round(total_unknown_province / total_beneficiaries, 2))

## Overview picture

In [20]:
# what we need
# - Total budget
# - Total beneficiaries
#   - target
#   - achieved
# Total female beneficiaries
#  - % of total
# Total youth beneficiaries
#  - % of total
# By section
#   LIV, RET, CRE
#   - target and achievement
#   - by department and phase

# we want a few things here:
# for Overview we want top level info (and OverviewSection) with a total value per phase per section
#
# and then for departments we want a dictionary of department_name to MultiMetricValue (2 values, 1 per phase)
programmes_by_type = {
    SectionEnum.job_opportunities.name: dict([(i, {}) for i in range(number_of_phases)]),
    SectionEnum.livelihoods.name: dict([(i, {}) for i in range(number_of_phases)]),
    SectionEnum.jobs_retain.name: dict([(i, {}) for i in range(number_of_phases)]),
}

achievements_by_type_by_month = {}
for section_type in [
    e.name for e in SectionEnum if e.name != "targets" and e.name != "budget_allocated"
]:
    achievements_by_type_by_month[section_type] = {}
    for month in months:
        achievements_by_type_by_month[section_type][month] = 0

# this stores the achievement values from the last months of the previous phase
achievement_cache = {
    SectionEnum.job_opportunities.name: dict([(i, 0) for i in range(number_of_phases-1)]),
    SectionEnum.livelihoods.name: dict([(i, 0) for i in range(number_of_phases-1)]),
    SectionEnum.jobs_retain.name: dict([(i, 0) for i in range(number_of_phases-1)]),
}

for department in all_data_departments:
    for phase_num, phase in enumerate(department.phases[:-1]):
        for section in phase.sections:
            for metric in section.metrics:
                for dimension in metric.dimensions:
                    if dimension.values and dimension.lookup == LookupTypeEnum.time.name:
                        # print("saving", section.section_type, dimension.values[-1].value)
                        achievement_cache[section.section_type][phase_num] += dimension.values[-1].value

for department in all_data_departments:
    for phase in department.phases:
        achievements_df = opportunity_achievements_df[phase.phase_num].iloc[3:, 1:].set_index(1)
        section_value = 0
        section_target_value = 0
        for section in phase.sections:
            if section.section_type == SectionEnum.targets.name:
                # skip over (budget and beneficiary) targets section
                continue
            total_value = 0
            total_target_value = 0
            for metric in section.metrics:
                #             if (
                #                 department.sheet_name == "DALRRD"
                #                 and metric.name == "Graduate Employment"
                #             ):
                #                 continue
                if phase.phase_num == 1 and section.section_type == SectionEnum.job_opportunities.name:
                    print(department.sheet_name, metric.value_target)
                if metric.name not in achievements_df.index:
                    print(
                        "Metric not found in achievements_df", department.name, metric.name
                    )
                total_value += metric.value
                if metric.value_target > 0:
                    total_target_value += metric.value_target
                for dimension in metric.dimensions:
                    if dimension.lookup == LookupTypeEnum.time.name:
                        for metric_value in dimension.values:
                            month = metric_value.key
                            value = metric_value.value
                            achievements_by_type_by_month[section.section_type][
                                month
                            ] += value
                            
            if (
                department.name == "Agriculture, Land Reform and Rural Development"
                and section.section_type == SectionEnum.livelihoods.name
            ):
                # this programme from DALRRD only has an overall target,
                # not one target per sub-programme
                if phase.phase_num == 0:
                    row = 8
                elif phase.phase_num == 1:
                    row = 7
                total_target_value = int(opportunity_targets_df[phase.phase_num].iloc[row, 2])
            elif (
                department.name == "Public Works and Infrastructure"
                and section.section_type == SectionEnum.job_opportunities.name
            ):
                # this is a phase 1 programme that just has an overall target
                total_target_value = int(opportunity_targets_df[phase.phase_num].iloc[47, 2])
            #         print(department.name, section.name, total_value, total_target_value)
            programmes_by_type[section.section_type][phase.phase_num][department.sheet_name] = {
                "value": total_value,
                "value_target": total_target_value,
                "phase": phase.phase_num
            }
            if "Total" not in programmes_by_type[section.section_type][phase.phase_num]:
                programmes_by_type[section.section_type][phase.phase_num]["Total"] = dict(
                    value=0, value_target=0
                )
            programmes_by_type[section.section_type][phase.phase_num]["Total"]["value"] += total_value
            programmes_by_type[section.section_type][phase.phase_num]["Total"][
                "value_target"
            ] += total_target_value
            section_value += total_value
            section_target_value += total_target_value

def in_phase(phase_num, month):
    if phase_num >= 0 and phase_num < len(phase_dates):
        (start_str, end_str) = phase_dates[phase_num]
        # print("got here", phase_num, month, int(start_str) <= int(month), int(month) <= int(end_str))
        if int(start_str) <= int(month) and int(month) <= int(end_str):
            return True
    return False

for section_type in achievements_by_type_by_month:
    if section_type == 'overview' or section_type == 'in_process':
        continue
    for month in achievements_by_type_by_month[section_type]:
        for phase_num in range(1, number_of_phases):
            # print(month, phase_num, in_phase(phase_num, month), in_phase(phase_num - 1, month))
            # print("test 2", in_phase(phase_num - 1, month))
            if in_phase(phase_num, month) and not in_phase(phase_num - 1, month):
                # print("adding cache to:", section_type, month,achievement_cache[section_type][phase_num - 1], "was", achievements_by_type_by_month[section_type][month])
                achievements_by_type_by_month[section_type][month] += achievement_cache[section_type][phase_num - 1]
# pp.pprint(programmes_by_type)

DBE 191392.0
DBE 95608.0
DCOGTA 11818.0
DEL 250.0
DFFE 2580.0
DFFE 1849.0
DFFE 1591.0
DFFE 2000.0
DFFE 130.0
DoH 2203.0
DoH 365.0
DHET 4500.0
DHET 3000.0
NT 35000.0
DSI 500.0
DSI 900.0
DSI 250.0
DSD 2000.0
DSD 1880.0
DSAC 35.0
DSAC 858.0
DSAC 21.0
Tourism 864.0
Tourism 200.0
DTIC 50000.0


In [21]:
achievement_totals_df = [pd.read_excel(consolidated_dashboard_excel, sheet_name='Demographic data', skiprows=2, usecols=range(2), nrows=3, names=['section', 'total'], index_col=0)]
achievement_totals_df.append(
    pd.read_excel(phase2_excel, sheet_name='Demographic data', skiprows=2, usecols=range(2), nrows=3, names=['section', 'total'], index_col=0)
)

# check targets for phase 1 - job opportunities
assert (
    programmes_by_type[SectionEnum.job_opportunities.name][0]["Total"]["value_target"]
    == opportunity_targets_df[0].iloc[6, 7]
), f'{SectionEnum.job_opportunities.name} total mismatch: {programmes_by_type[SectionEnum.job_opportunities.name][0]["Total"]["value_target"]} vs {opportunity_targets_df[0].iloc[6, 7]}'

# check targets for phase 2 - job opportunities
assert (
    programmes_by_type[SectionEnum.job_opportunities.name][1]["Total"]["value_target"]
    == opportunity_targets_df[1].iloc[5, 7]
), f'{SectionEnum.job_opportunities.name} total mismatch: {programmes_by_type[SectionEnum.job_opportunities.name][1]["Total"]["value_target"]} vs {opportunity_targets_df[1].iloc[5, 7]}'


In [22]:
# check achievements for phase 1 - job opportunities
assert (
    programmes_by_type[SectionEnum.job_opportunities.name][0]["Total"]["value"] == achievement_totals_df[0].loc["Jobs created","total"]
), f'{SectionEnum.job_opportunities.name} total mismatch {programmes_by_type[SectionEnum.job_opportunities.name][0]["Total"]["value"]} vs {achievement_totals_df[0].loc["Jobs created"]}'

# check achiements for phase 2 - job opportunities
assert (
    programmes_by_type[SectionEnum.job_opportunities.name][1]["Total"]["value"] == achievement_totals_df[1].loc["Jobs created","total"]
), f'{SectionEnum.job_opportunities.name} total mismatch {programmes_by_type[SectionEnum.job_opportunities.name][1]["Total"]["value"]} vs {achievement_totals_df[1].loc["Jobs created"]}'


In [23]:
# check targets for phase 1 - livelihoods support
assert (
    programmes_by_type[SectionEnum.livelihoods.name][0]["Total"]["value_target"]
    == opportunity_targets_df[0].iloc[7, 7]
), f'{SectionEnum.livelihoods.name} total mismatch: {programmes_by_type[SectionEnum.livelihoods.name][0]["Total"]["value_target"]} vs {opportunity_targets_df[0].iloc[7, 7]}'

# check targets for phase 2 - livelihoods support
assert (
    programmes_by_type[SectionEnum.livelihoods.name][1]["Total"]["value_target"]
    == opportunity_targets_df[1].iloc[6, 7]
), f'{SectionEnum.livelihoods.name} total mismatch: {programmes_by_type[SectionEnum.livelihoods.name][1]["Total"]["value_target"]} vs {opportunity_targets_df[1].iloc[6, 7]}'


# check achievements for phase 1 - livelihoods support
assert (
    programmes_by_type[SectionEnum.livelihoods.name][0]["Total"]["value"] == achievement_totals_df[0].loc["Livelihoods supported","total"]
), f'{SectionEnum.job_opportunities.name} total mismatch {programmes_by_type[SectionEnum.livelihoods.name][0]["Total"]["value"]} vs {achievement_totals_df[0].loc["Livelihoods supported"]}'

# check achievements for phase 2 - livelihoods support
assert (
    programmes_by_type[SectionEnum.livelihoods.name][1]["Total"]["value"] == achievement_totals_df[1].loc["Livelihoods supported","total"]
), f'{SectionEnum.job_opportunities.name} total mismatch {programmes_by_type[SectionEnum.livelihoods.name][1]["Total"]["value"]} vs {achievement_totals_df[1].loc["Livelihoods supported"]}'


In [24]:
# check targets for phase 1 - jobs retained
assert (
    programmes_by_type[SectionEnum.jobs_retain.name][0]["Total"]["value_target"]
    == opportunity_targets_df[0].iloc[8, 7]
), f'{SectionEnum.jobs_retain.name} total mismatch: {programmes_by_type[SectionEnum.jobs_retain.name][0]["Total"]["value_target"]} vs {opportunity_targets_df[0].iloc[8, 7]}'

# check achivements for phase 2 - jobs retained
assert (
    programmes_by_type[SectionEnum.jobs_retain.name][0]["Total"]["value"] == achievement_totals_df[0].loc["Jobs retained","total"]
), f'{SectionEnum.job_opportunities.name} total mismatch {programmes_by_type[SectionEnum.jobs_retain.name][0]["Total"]["value"]} vs {achievement_totals_df[0].loc["Jobs retained"]}'


In [25]:
# we currently have programmes_by_type which is structured 
# programme_type -> phase -> department
# 
# we need programme_type -> department -> phase 
# with phase_num starting at 1

programmes_by_type_summarised = {}
for programme_type in programmes_by_type:
    programmes_by_type_summarised[programme_type] = {}
    for phase_num in programmes_by_type[programme_type]:
        for department in programmes_by_type[programme_type][phase_num]:
            if department not in programmes_by_type_summarised[programme_type]:
                programmes_by_type_summarised[programme_type][department] = {'value': {}, 'value_target': {}}
            programmes_by_type_summarised[programme_type][department]["value"][phase_num+1] = programmes_by_type[programme_type][phase_num][department]["value"]
            programmes_by_type_summarised[programme_type][department]["value_target"][phase_num+1] = programmes_by_type[programme_type][phase_num][department]["value_target"]
pp.pprint(programmes_by_type_summarised)

{ 'job_opportunities': { 'DALRRD': { 'value': {1: 1464, 2: 0},
                                     'value_target': {1: 0, 2: 0}},
                         'DBE': { 'value': {1: 319482, 2: 276627},
                                  'value_target': {1: 300000.0, 2: 287000.0}},
                         'DCDT': {'value': {2: 0}, 'value_target': {2: 0}},
                         'DCOGTA': { 'value': {1: 5560, 2: 7304},
                                     'value_target': {1: 25000.0, 2: 11818.0}},
                         'DEL': {'value': {2: 0}, 'value_target': {2: 250.0}},
                         'DFFE': { 'value': {1: 23943, 2: 4549},
                                   'value_target': {1: 50311.0, 2: 8150.0}},
                         'DHET': {'value': {2: 0}, 'value_target': {2: 7500.0}},
                         'DPWI': { 'value': {1: 1881},
                                   'value_target': {1: 1560}},
                         'DSAC': { 'value': {1: 1461, 2: 0},
                    

In [26]:
pp.pprint(achievements_by_type_by_month)

{ 'in_process': { '202010': 0,
                  '202011': 0,
                  '202012': 0,
                  '202101': 0,
                  '202102': 0,
                  '202103': 0,
                  '202104': 0,
                  '202105': 0,
                  '202106': 0,
                  '202107': 0,
                  '202108': 0,
                  '202109': 0,
                  '202110': 0,
                  '202111': 0,
                  '202112': 0,
                  '202201': 0},
  'job_opportunities': { '202010': 1982,
                         '202011': 219399,
                         '202012': 295159,
                         '202101': 322414,
                         '202102': 362212,
                         '202103': 374250,
                         '202104': 374484,
                         '202105': 374484,
                         '202106': 374484,
                         '202107': 374484,
                         '202108': 374484,
                         '202109

In [27]:
def sort_dept_metric(element):
    # element is a pair - (department_name, dictionary) where the dictionary contains
    # {'value': {phase_num: amount} ... }
    value = element[1]['value']
    value_total = 0
    for phase_num in value:
        value_total += value[phase_num]
    return value_total
    
breakdown_metrics = [
                PhasedMetric(
                    name=section_titles[section_name],
                    metric_type=section_name,
                    viz=VizTypeEnum.full.name,
                    total_value=sum(programmes_by_type_summarised[section_name]["Total"]["value"].values()),
                    total_value_target=sum(programmes_by_type_summarised[section_name]["Total"]["value_target"].values()),
                    value=programmes_by_type_summarised[section_name]["Total"]["value"],
                    value_target=programmes_by_type_summarised[section_name]["Total"][
                        "value_target"
                    ],
                    dimensions=[
                        Dimension(
                            # by department needs to have a MultiMetricValue
                            name="by department",
                            viz=VizTypeEnum.bar.name,
                            lookup=LookupTypeEnum.department.name,
                            values=[
                                MultiMetricValue(
                                    key=department_name,
                                    value=outputs["value"],
                                    value_target=outputs["value_target"],
                                )
                                for department_name, outputs in sorted(
                                    department_info.items(),
                                    key=sort_dept_metric,
                                    reverse=True,
                                )
                                if not (
                                    department_name.startswith("value")
                                    or department_name == "Total"
                                )
                            ],
                        ),
                        Dimension(
                            # by time needs to have PhasedMetricValues - currently this is broken 
                            # because I don't know how to handle overlapping phases
                            name="over time",
                            viz=VizTypeEnum.line.name,
                            lookup=LookupTypeEnum.time.name,
                            values=[
                                MetricValue(key=key, value=value)
                                for key, value in achievements_by_type_by_month[
                                    section_name
                                ].items()
                            ],
                        ),
                    ],
                )
                for section_name, department_info in programmes_by_type_summarised.items()
                if not section_name.startswith("value")
            ]
# add up all the metrics across all the programmes
current_targets = dict([
    (phase_num, sum([metric.value[phase_num] if metric.value[phase_num] > 0 else 0 for metric in breakdown_metrics]))
    for phase_num in range(1,number_of_phases+1)])
current_achievements = dict([
    (phase_num, sum([metric.value_target[phase_num] if metric.value_target[phase_num] > 0 else 0 for metric in breakdown_metrics]))
    for phase_num in range(1,number_of_phases+1)])


In [28]:
# metrics breakdown

female_by_phases = dict(
    [(phase_num+1, total_female[phase_num] / (total_beneficiaries[phase_num] - total_unknown_gender[phase_num])) 
     for phase_num in range(number_of_phases)]
)
overall_female_perc = sum(female_by_phases.values()) / 2

overview_metrics = []

phase_1_budget = opportunity_targets_df[0].iloc[2,6]
phase_2_budget = opportunity_targets_df[1].iloc[2,6]

total_budget = PhasedMetric(
    name="Total budget allocated",
    metric_type=MetricTypeEnum.currency.value,
    viz=VizTypeEnum.full.name,
    total_value=phase_1_budget + phase_2_budget,
    value=[phase_1_budget, phase_2_budget],
    dimensions=[],
    
)
overview_metrics.append(total_budget)

achievements_by_phase_value = dict([(phase_num, 0) for phase_num in range(number_of_phases)])
achievements_by_phase_value_target = dict([(phase_num, 0) for phase_num in range(number_of_phases)])

for section_type in programmes_by_type:
    for phase_num in range(number_of_phases):
        # print(section_type, programmes_by_type[section_type][phase_num]['Total'])
        achievements_by_phase_value[phase_num] += programmes_by_type[section_type][phase_num]['Total']['value']
        achievements_by_phase_value_target[phase_num] += programmes_by_type[section_type][phase_num]['Total']['value_target']

achievements = PhasedMetric(
    name="Total beneficiaries assisted",
    metric_type=MetricTypeEnum.count.value,
    viz=VizTypeEnum.full.name,
    total_value=sum(achievements_by_phase_value.values()),
    value=[achievements_by_phase_value[0], achievements_by_phase_value[1]],
    total_value_target=sum(achievements_by_phase_value_target.values()),
    value_target=[achievements_by_phase_value_target[0], achievements_by_phase_value_target[1]],
    dimensions=[]
)
overview_metrics.append(achievements)

gender_breakdown = PhasedMetric(
    name="Total female beneficiaries",
    metric_type="targets_count",
    viz=VizTypeEnum.compact.name,
    value=female_by_phases,
    total_value=overall_female_perc,
    value_target=-1,
    dimensions=[]
)
overview_metrics.append(gender_breakdown)

youth_by_phases=value=dict(
    [(phase_num+1, total_youth[phase_num] / (total_beneficiaries[phase_num] - total_unknown_youth[phase_num]))
    for phase_num in range(number_of_phases)]
)
overall_youth_perc = sum(youth_by_phases.values()) / 2
youth_breakdown = PhasedMetric(
    name="Total youth beneficiaries",
    metric_type="targets_count",
    viz=VizTypeEnum.compact.name,
    value=youth_by_phases,
    total_value=overall_youth_perc,
    value_target=-1,
    dimensions=[]
)
overview_metrics.append(youth_breakdown)
                # Metric(
                #         name="Beneficiaries by province",
                #         metric_type="targets_count",
                #         value=-1,
                #         value_target=-1,
                #         dimensions=[
                #             Dimension(
                #                 name="Beneficiaries by province",
                #                 viz=VizTypeEnum.bar.name,
                #                 lookup=LookupTypeEnum.province.name,
                #                 values=[
                #                     MultiMetricValue(key=abbrev, value=
                #                                      dict(
                #                                          [(phase_num, total_provincial[abbrev][phase_num]) 
                #                                           for phase_num in range(number_of_phases)]
                #                                     )) for abbrev in province_abbreviations
                #                 ],
                #                 data_missing=False
                #             )                        
                #         ]
                #     )
            # ])

            

In [29]:
# assert (
#     opportunity_achievements_df.iloc[6:59, total_achievement_column].sum()
#     == opportunity_achievements_df.iloc[59, total_achievement_column]
# ), "Sum of achievements does not add up to reported total"
# assert (
#     opportunity_targets_df.iloc[2:56, 2].sum() == opportunity_targets_df.iloc[56, 2]
# ), "Sum of targets does not add up to reported total"

In [30]:


month_info = description_df.loc["Overview", "Data captured until"]
try:
    month = month_info.strftime('%Y%m')
except AttributeError as e:
    month_parts = month_info.split('-')
    month = month_parts[2] + month_parts[1]
overview = Overview(
    month=month,
    name="Programme overview",
    lead=leads["Overview"],
    phase_dates=phase_dates,
    paragraph=paragraphs["Overview"],
    footer_header=leads["Disclaimer"],
    footer_paragraph=paragraphs["Disclaimer"],
    sections=[
        Section(
            name="Overall Achievements",
            section_type=SectionEnum.overview.name,
            metrics=overview_metrics,
            value=current_targets,
            value_target=current_achievements,
        ),
        Section(
            name="Performance breakdown",
            section_type=SectionEnum.overview.name,
            metrics=breakdown_metrics,
            value=[-1,-1],
            value_target=[-1,-1]
        ),
    ],
)

# overview.sections.insert(
#     0,
#     Section(
#         name=section_titles[SectionEnum.targets.name + "_overview"],
#         section_type=SectionEnum.targets.name,
#         metrics=[
#             Metric(
#                 name=metric_titles[SectionEnum.targets.name][
#                     MetricTypeEnum.currency.name
#                 ],
#                 metric_type=MetricTypeEnum.currency.name,
#                 dimensions=[],
#                 # value=int(opportunity_targets_df.iloc[2, 7] * 1000),
#                 value=0,
#                 value_target=(opportunity_targets_df.iloc[2, 6] * 1000),
#             ),
#             Metric(
#                 name=metric_titles[SectionEnum.targets.name][MetricTypeEnum.count.name],
#                 metric_type=MetricTypeEnum.count.name,
#                 dimensions=[],
#                 value=int(
#                     opportunity_achievements_df.iloc[59, total_achievement_column]
#                 ),
#                 value_target=int(opportunity_targets_df.iloc[56, 2]),
#             ),
#             Metric(
#                 name="Opportunities in process",
#                 metric_type=MetricTypeEnum.count.name,
#                 dimensions=[],
#                 value_target=int(opportunity_achievements_df.iloc[2, 1]),
#                 value=0,
#             ),
#         ],
#         value=None,
#         value_target=None,
#     ),
# )

all_data = Everything(
    overview=overview,
    departments=all_data_departments
)
# # print(overview.to_json(indent=2))

In [31]:
# ad_set = set()
# for department in all_data.departments:
#     for section in department.sections:
#         if section.section_type == SectionEnum.targets.name:
#             continue
#         for metric in section.metrics:
#             ad_set.add(metric.name)

# ot_set = set(list(opportunity_targets_df.iloc[2:55, 1]))
# imp_set = set(list(implementation_status_df.iloc[:53, 1]))
# ot_set.difference(ad_set)

# Save final data

In [32]:
output_filename = output_dir + "/all_data.json"
all_data.departments.sort(key=operator.attrgetter("sheet_name"))
open(output_filename, "w").write(all_data.to_json(indent=2))
# print(all_data.to_json(indent=2))
print("DONE")

DONE


In [33]:
# programme_status_df = pd.read_excel(
#     mar_opportunities_excel, sheet_name="Implementation status", header=None
# )

# to_camel_case = lambda match: match.group(1) + match.group(2).upper() + match.group(3)

# [
#     re.sub(r"(\S*) (\w)(.*)", to_camel_case, status)
#     for status in implementation_status_df.iloc[3:, 2].dropna().unique()
# ]

In [34]:
# for dept in all_data.departments:
#     print(f"\t'{dept.name}': '{dept.sheet_name}',")

In [35]:
# json.dump(metric_titles, open(output_dir + "/metric_titles.json", "w"), indent=2)